In [1]:
import pandas as pd
import numpy as np
import requests
import json
from bs4 import BeautifulSoup

## 코드 짜보기

In [2]:
url  = 'https://movie.naver.com/movie/running/current.nhn#'

In [3]:
# 위 url 에서 노출되는 영화는 89건
response  = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
found_items = soup.select('div.lst_wrap > ul.lst_detail_t1 > li')
len(found_items)

96

영화의 상세 페이지에도 크롤링할 정보가 모두 노출되어 있기에 다음과 같이 진행
1. 크롤링 할 영화의 상세 페이지 url 저장
2. 각 영화의 상세 페이지 진입 후 각 정보 크롤링

In [4]:
url  = 'https://movie.naver.com/movie/running/current.nhn#'

# 위 url 에서 노출되는 영화는 89건
response  = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
found_items = soup.select('div.lst_wrap > ul.lst_detail_t1 > li')
found_items = found_items[:5] # 5개로 제한
data = []
for item in found_items:
    
    # 영화 상세 페이지 url
    movie_url = 'https://movie.naver.com' + item.select_one('dl.lst_dsc > dt.tit > a')['href']
    movie_url = movie_url.replace('basic', 'point') # 영화 상세 메인 페이지 -> 영화 상세 평점 페이지 (리뷰 개수 때문에))
    code = movie_url.split('code=')[1] # 영화 코드 추출

    # 상세 페이지로 이동
    response  = requests.get(movie_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # 영화 제목
    title = soup.select_one('div.mv_info_area > div.mv_info > h3.h_movie > a').text
    
    # 주연 배우 3인
    actors = soup.select('div.mv_info_area > div.mv_info > dl.info_spec > dd:nth-child(6) > p > a')
    actors_list = []
    for actor in actors:
        actors_list.append(actor.text)
        
    # 평점들
    a_score = soup.select_one('div.mv_info_area > div.mv_info > div.main_score > div.score > a > div > span').text
    a_score = a_score.split('점')[1]
    
    r_score = soup.select_one('div.mv_info_area > div.mv_info > div.main_score > div.score:nth-child(2) > div.spc_score_area > a > div > span > span')['style'] # 평점이 별점 css의 넓이로도 표현됨
    r_score = int(r_score.split(':')[1].split('%')[0].replace('.', '')) / 100 # 넓이를 int로 변환 후 평점으로 변경
    
    n_score = soup.select_one('div.mv_info_area > div.mv_info > div.main_score > div.score_left > div.star_score > a > span > span')['style']
    n_score = int(n_score.split(':')[1].split('%')[0].replace('.', '')) / 100
    
    # 관람객 리뷰
    review_list = []
    for i in range(1,3): # 리뷰 20개 이므로 2페이지까지
        reviews_url = f'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code={code}&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page={i}'
        response = requests.get(reviews_url)
        soup = BeautifulSoup(response.text, 'html.parser')
        reviews = soup.select('div.score_result > ul > li')
        for review in reviews:
            score = review.select_one('div.star_score > em').text
            try:
                nick = review.select_one('div.score_reple > dl > dt > em > a > span').text
            except:
                nicl = 'none'
            content = review.select_one('div.score_reple > p > span').text.strip()
            
            value = [score, nick, content]
            review_list.append(value)
    print([title], '크롤링 완료')
    value = [title, actors_list, a_score, r_score, n_score, review_list]
    data.append(value)

['인비저블맨'] 크롤링 완료
['1917'] 크롤링 완료
['지푸라기라도 잡고 싶은 짐승들'] 크롤링 완료
['작은 아씨들'] 크롤링 완료


TypeError: 'NoneType' object is not subscriptable

## 함수로 만들기

In [213]:
def getMovie(url, each, page): # each 는 크롤링할 영화의 개수를 의미 여기서는 5개로 주어짐, page 는 크롤링할 리뷰 페이지를 의미 (페이지당 10개))

    # 위 url 에서 노출되는 영화는 89건
    response  = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    found_items = soup.select('div.lst_wrap > ul.lst_detail_t1 > li')
    found_items = found_items[: each]
    data = []
    for item in found_items:

        # 영화 상세 페이지 url
        movie_url = 'https://movie.naver.com' + item.select_one('dl.lst_dsc > dt.tit > a')['href']
        movie_url = movie_url.replace('basic', 'point') # 영화 상세 메인 페이지 -> 영화 상세 평점 페이지 (리뷰 개수 때문에))
        code = movie_url.split('code=')[1] # 영화 코드 추출

        # 상세 페이지로 이동
        response  = requests.get(movie_url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # 영화 제목
        title = soup.select_one('div.mv_info_area > div.mv_info > h3.h_movie > a').text

        # 주연 배우 3인
        actors = soup.select('div.mv_info_area > div.mv_info > dl.info_spec > dd:nth-child(6) > p > a')
        actors_list = []
        for actor in actors:
            actors_list.append(actor.text)

        # 평점들 (관람객평점, 기자/평론가평점, 네티즌평점)
        try:
            a_score = soup.select_one('div.mv_info_area > div.mv_info > div.main_score > div.score > a > div > span').text
            a_score = a_score.split('점')[1]
            if a_score == ' 없음':
                a_score = 'none'
        except:
            a_score = 'none'

        try:
            r_score = soup.select_one('div.mv_info_area > div.mv_info > div.main_score > div.score:nth-child(2) > div.spc_score_area > a > div > span > span')['style'] # 평점이 별점 css의 넓이로도 표현됨
            r_score = int(r_score.split(':')[1].split('%')[0].replace('.', '')) / 100 # 넓이를 int로 변환 후 평점으로 변경
        except:
            r_score = 'none'
            
        try:
            n_score = soup.select_one('div.mv_info_area > div.mv_info > div.main_score > div.score_left > div.star_score > a > span > span')['style']
            n_score = int(n_score.split(':')[1].split('%')[0].replace('.', '')) / 100
        except:
            r_score = 'none'
            
        # 관람객 리뷰
        review_list = []
        for i in range(1,page): # page는 크롤링할 리뷰페이지를 의미, 여기서는 리뷰 20개 이므로 2페이지까지( 1page x 10 )
            # {code} 는 위에서 크롤링한 영화 고유의 code를 의미함
            #reviews_url 은 리뷰데이터를 따로 보관하고 있는 페이지임
            reviews_url = f'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code={code}&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page={i}'
            response = requests.get(reviews_url)
            soup = BeautifulSoup(response.text, 'html.parser')
            reviews = soup.select('div.score_result > ul > li')
            for review in reviews:
                score = review.select_one('div.star_score > em').text
                try:
                    nick = review.select_one('div.score_reple > dl > dt > em > a > span').text
                except:
                    nicl = 'none'
                content = review.select_one('div.score_reple > p > span').text.strip()

                value = [score, nick, content]
                review_list.append(value)
        print([title], '크롤링 완료')
        value = [title, actors_list, a_score, r_score, n_score, review_list]
        data.append(value)
    return data

In [201]:
url = 'https://movie.naver.com/movie/running/current.nhn'
data = getMovie(url, 5, 2) # 영화 5개 정보 및 각 영화당 2페이지 리뷰(2*10)크롤링

['지푸라기라도 잡고 싶은 짐승들'] 크롤링 완료
['정직한 후보'] 크롤링 완료
['1917'] 크롤링 완료
['작은 아씨들'] 크롤링 완료
['클로젯'] 크롤링 완료


In [196]:
df = pd.DataFrame(data, columns = ['영화제목', '주연배우', '관람객평점', '기자/평론가평점', '네티즌평점', '리뷰'])

In [197]:
df.head()

,영화제목,주연배우,관람객평점,기자/평론가평점,네티즌평점,리뷰
0,지푸라기라도 잡고 싶은 짐승들,"[전도연, 정우성, 배성우]",9.13,6.80,7.68,"[[10, bohemian(mabu****), 난 전도연의 화류계 캐릭터가 좋다. ..."
1,정직한 후보,"[라미란, 김무열, 나문희]",8.74,5.38,7.77,"[[6, lililli(wken****), 관람객], [10, 김다솜(kmjd***..."
2,1917,"[조지 맥케이, 딘-찰스 채프먼]",9.67,7.33,9.13,"[[8, hose(jsd9****), 충무로: 이거 어케하는거냐?], [10, 6월..."
3,작은 아씨들,"[시얼샤 로넌, 엠마 왓슨, 플로렌스 퓨]",9.27,8.00,8.93,"[[10, 문덕현(mdh0****), 재밌다고 느끼면 추천을 아니다 비추천], [1..."
4,클로젯,"[하정우, 김남길, 허율]",8.47,5.50,6.88,"[[10, 깜땀(qw13****), 김남길 미치게 섹시하다 진심], [10, gil..."


In [214]:
# 30개 영화와 각 영화당 50개 리뷰씩 크롤링
data2 = getMovie(url, 20, 5)

['지푸라기라도 잡고 싶은 짐승들'] 크롤링 완료
['정직한 후보'] 크롤링 완료
['1917'] 크롤링 완료
['작은 아씨들'] 크롤링 완료
['클로젯'] 크롤링 완료
['기생충'] 크롤링 완료
['수퍼 소닉'] 크롤링 완료
['하이, 젝시'] 크롤링 완료
['숀더쉽 더 무비: 꼬마 외계인 룰라!'] 크롤링 완료
['조조 래빗'] 크롤링 완료
['남산의 부장들'] 크롤링 완료
['버즈 오브 프레이(할리 퀸의 황홀한 해방)'] 크롤링 완료
['극장판 미니특공대: 공룡왕 디노'] 크롤링 완료
['타오르는 여인의 초상'] 크롤링 완료
['극장판 원피스 스탬피드'] 크롤링 완료
['이 멋진 세계에 축복을! 붉은 전설'] 크롤링 완료
['작가 미상'] 크롤링 완료
['페인 앤 글로리'] 크롤링 완료
['사마에게'] 크롤링 완료
['인셉션'] 크롤링 완료


In [215]:
df = pd.DataFrame(data2, columns = ['영화제목', '주연배우', '관람객평점', '기자/평론가평점', '네티즌평점', '리뷰'])

In [216]:
df

,영화제목,주연배우,관람객평점,기자/평론가평점,네티즌평점,리뷰
0,지푸라기라도 잡고 싶은 짐승들,"[전도연, 정우성, 배성우]",9.13,6.83,7.68,"[[10, bohemian(mabu****), 난 전도연의 화류계 캐릭터가 좋다. ..."
1,정직한 후보,"[라미란, 김무열, 나문희]",8.74,5.38,7.77,"[[6, lililli(wken****), 관람객], [10, 김다솜(kmjd***..."
2,1917,"[조지 맥케이, 딘-찰스 채프먼]",9.67,7.67,9.12,"[[8, hose(jsd9****), 충무로: 이거 어케하는거냐?], [10, 6월..."
3,작은 아씨들,"[시얼샤 로넌, 엠마 왓슨, 플로렌스 퓨]",9.27,8,8.93,"[[10, 문덕현(mdh0****), 재밌다고 느끼면 추천을 아니다 비추천], [1..."
4,클로젯,"[하정우, 김남길, 허율]",8.47,5.5,6.88,"[[10, 깜땀(qw13****), 김남길 미치게 섹시하다 진심], [10, gil..."
5,기생충,"[송강호, 이선균, 조여정]",9.07,9.06,8.49,"[[10, brilliant_AKA_밝음(bril****), 비에 젖지 않는 고급 ..."
6,수퍼 소닉,"[짐 캐리, 제임스 마스던, 벤 슈와츠]",8.66,5,8.44,"[[9, STARK(lshc****), 소닉 성형이 신의 한수... 제작진이 성형시..."
7,"하이, 젝시","[아담 드바인, 로즈 번]",9.33,5.5,6.80,"[[6, napa****, 그냥저냥 킬링타임용. 이럴거면 아예 청불 등급에 더 쎈 ..."
8,숀더쉽 더 무비: 꼬마 외계인 룰라!,"[저스틴 플레쳐, 아멜리아 비테일]",10.00,6.75,9.57,"[[10, JINs(sjho****), 자녀들과 같이 보기에 아주 건전하고 재미있는..."
9,조조 래빗,"[스칼렛 요한슨, 로만 그리핀 데이비스, 타이카 와이티티]",9.08,7.17,9.27,"[[10, 2023032(yjpa****), 연기, 연출, 스토리, 영상미 뭐 하나..."
